In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-onboarding"

In [7]:
import os
import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN_API_KEY: ")
os.environ["TAVIS_API_KEY"] = getpass.getpass("TAVIS_API_KEY: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY: ")

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
model = ChatGroq(model_name="llama3-8b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )

False

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [
    TavilySearchResults(
        max_results=1
    )
]

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0,
    streaming=True,
)

agent_runnable = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)


In [2]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input: str
    chat_history: List[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    

Define the node

In [3]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolNode



tool_executor = ToolNode(tools)

# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome} 



def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"
    

    

# Define the graph

In [4]:
from langgraph.graph import END, StateGraph

tool_node = ToolNode(tools=tools)

workflow = StateGraph(AgentState)
workflow.add_node("agent", run_agent)
workflow.add_node("action", tool_node)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")
app = workflow.compile()


In [6]:
inputs = {"input": "The 1st president of America", "chat_history": []}

for s in app.stream(inputs):
    print(list(s.values())[0])  
    print("----")
    

{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----


In [4]:
load_dotenv()
# Create a ChatOpenAI model
#model = ChatOpenAI(model="gpt-4o")
model = ChatGroq(model_name="llama3-70b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )
result = model.invoke("81 chia 9 bằng bao nhiêu?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

Full result:
content='81 chia 9 bằng 9.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_time': 0.028300789, 'prompt_time': 0.000255537, 'queue_time': 0.220509222, 'total_time': 0.028556326}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None} id='run--ccdb18f4-12ad-4643-bcea-d4afee2999ce-0' usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27}
Content only:
81 chia 9 bằng 9.


In [14]:
from dotenv import dotenv_values

env_vars = dotenv_values()  # Trả về dict chỉ chứa các biến từ file .env
for key, value in env_vars.items():
    print(f"{key} = {value}")

LANGCHAIN_API_KEY = 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
GROQ_API_KEY = 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
TAVILY_API_KEY = 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
